In [ ]:
# pip install keras
# pip install tensorflow

In [ ]:
import pandas as pd
import re
import random
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
from keras import optimizers
import keras
from keras import layers
from nltk.tokenize import sent_tokenize,word_tokenize
import time

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Preprocessed Data

In [ ]:
review = {}
scores=pd.read_csv("/content/drive/My Drive/NLP/review_with_scores.csv")
scores=scores.sort_values("automated_readability_index",ascending=False)
review['preprocessed']=scores["text"]
review['preprocessed'].reset_index(inplace=True,drop=True)
review['preprocessed']=review['preprocessed'].apply(lambda x: "<SOR>"+x+"<EOR>")

In [ ]:
#pick the top N high readibility to train
sample_size=8000
review['sample'] = review['preprocessed4'][0:sample_size] 

# Create character-level Corpus

In [ ]:
filename = 'short_reviews.txt'
review['sample'].to_csv(filename, header=None, index=None, sep=' ')
train_text = open('short_reviews.txt').read()
print('Corpus length:', len(train_text))

Corpus length: 2163801


In [ ]:
# set default value for key parameters
maxlen=60
step=1
chars = ['\n',' ','!', '$', "'",  ',', '.', '?', '@', '"',
             '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 
             'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 
             'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 
             'U', 'V', 'W', 'X', 'Y', 'Z', 
             'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
             'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 
             'u', 'v', 'w', 'x', 'y', 'z','<','>']
             
print('Unique characters:', len(chars))

#create dictionary
char_indices = {}
for char in chars:
  char_indices[char]=chars.index(char)

Unique characters: 74


# Train Model

In [ ]:
from keras import models
import tensorflow as tf

## Load the model
8000_Best_Model_RNN.hdf5 saved from the model below and it took 7hrs to run

In [ ]:
model = tf.keras.models.load_model("/content/drive/My Drive/NLP/8000_Best_Model_RNN.hdf5")

## RNN Model

In [ ]:
#Model
model = keras.models.Sequential()
model.add(layers.LSTM(1024, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(layers.LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.Adam(learning_rate=0.001) # default setting: learning_rate=0.001
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
filepath="/content/drive/My Drive/NLP/8000_Best_Model_RNN.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')
reduce_lr = ReduceLROnPlateau(monitor='loss', 
                              factor=0.5,
                              patience=1, 
                              min_lr=0.00001) # default setting: min_lr=0.00001
callbacks_list = [checkpoint, reduce_lr]

In [ ]:
#necessary to process large data
#X : tensor 把text split to len(textchunk)-60 files --> 60個character為單位去拆分
#txtChunk: text file
def getChunk(txtChunk, maxlen=60, step=1):
    sentences = []
    next_chars = []
    for i in range(0, len(txtChunk) - maxlen, step):
        sentences.append(txtChunk[i : i + maxlen])
        next_chars.append(txtChunk[i + maxlen])
    print('number of sequences:', len(sentences))
    print('Vectorization...')
    X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
            y[i, char_indices[next_chars[i]]] = 1
    return [X, y]

In [ ]:
t0 = time.time()
for iteration in range(1, 20):
    print('Iteration', iteration)
    with open("short_reviews.txt") as f:
        for chunk in iter(lambda: f.read(90000), ""): #split entire text to many chunks and each one has 90000 chars 
            X, y = getChunk(chunk)
            model.fit(X, y, batch_size=128, epochs=1, callbacks=callbacks_list)
 t1 = time.time()           
 print("Training took {:.2f}s".format(t1 - t0))

Iteration 1
number of sequences: 89940
Vectorization...
703/703 [==============================] - 71s 102ms/step - loss: 0.5961

Epoch 00001: loss improved from inf to 0.59610, saving model to /content/drive/My Drive/NLP/8000_keep_Best_Model_RNN.hdf5
number of sequences: 89940
Vectorization...
703/703 [==============================] - 71s 102ms/step - loss: 0.5630

Epoch 00001: loss improved from 0.59610 to 0.56298, saving model to /content/drive/My Drive/NLP/8000_keep_Best_Model_RNN.hdf5
number of sequences: 89940
Vectorization...
703/703 [==============================] - 71s 102ms/step - loss: 0.5460

Epoch 00001: loss improved from 0.56298 to 0.54603, saving model to /content/drive/My Drive/NLP/8000_keep_Best_Model_RNN.hdf5
number of sequences: 89940
Vectorization...
703/703 [==============================] - 71s 101ms/step - loss: 0.5416

Epoch 00001: loss improved from 0.54603 to 0.54162, saving model to /content/drive/My Drive/NLP/8000_keep_Best_Model_RNN.hdf5
number of sequen

# Review Generation

In [ ]:
def generate_reviews_v2(temperature = 0.75, maxlen = 60, len_review=60):
    seed_text = ''
    while len(seed_text) < 80:
      seed_text += review['trained'][random.randint(0, len(review['trained']))] 
    start_index = np.random.randint(0, len(seed_text) - maxlen - 1)
    generated_text = seed_text[start_index: start_index + maxlen]
    output = seed_text[: start_index + maxlen]
    word = ''

    for i in range(len_review):
      
      sampled = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
          sampled[0, t, char_indices[char]] = 1.
      
      preds = model.predict(sampled, verbose=0)[0]
      preds = np.asarray(preds).astype('float64')
      preds = np.log(preds) / temperature
      exp_preds = np.exp(preds)
      preds = exp_preds / np.sum(exp_preds)
      probas = np.random.multinomial(1, preds, 1)
      next_index = np.argmax(probas)
      next_char = chars[next_index]
      output += next_char
      generated_text = output[-60:]
    
    return output

In [ ]:
output=generate_reviews_v2(temperature = 0.75, maxlen = 60, len_review=100)
output

'<SOR>We had a company dinner at rosewood and it could not have been a better experience . Location from our hotel downtown was easily accessible on scooters as well as uber , the activities available on the grass as our large group arrived was a huge plus and the staff was kind and accommodating to each and charge us promptly and helped me kind of food to put it until I actuall'

In [ ]:
output=generate_reviews_v2(temperature = 0.75, maxlen = 60, len_review=60)
output

'<SOR>This place has incredible food and I am pretty sure that many of the negative reviews here are at least a little bit racist . The soup dumplings are amazing , and I would recommend the eggplant and the chinese broccoli or sauted greens to accompany whatever heavier dishes have the thing they have tastes like the eggplant , bl'

In [ ]:
output=generate_reviews_v2(temperature = 0.75, maxlen = 60, len_review=30)
output

'<SOR>Absolutely amazing french food , never disappointing . We started with escargot and oysters beautiful way to get the appetite going , with some fresh bread and a cocktail . Our server was friendly , courteous and knowledgeable but not pompone establishments when they ra'